In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import plotly.express as px
from matplotlib import colors
from os import listdir
from os.path import isfile, join
import numpy as np

In [2]:
table = input('Enter the name of the table: ')

In [3]:
soy_trade = pd.read_csv('https://raw.githubusercontent.com/friedrich-henrique/datasets_soybeans_research/main/soytrade.csv')
flour_trade = pd.read_csv('https://raw.githubusercontent.com/friedrich-henrique/datasets_soybeans_research/main/flourtrade.csv')
country_size = pd.read_csv('https://raw.githubusercontent.com/friedrich-henrique/datasets_soybeans_research/main/countries_size.csv')
ex_rate = pd.read_csv('https://raw.githubusercontent.com/friedrich-henrique/datasets_soybeans_research/main/Exchange%20Rates.csv', skiprows=2)
gdp_data = pd.read_csv('https://raw.githubusercontent.com/friedrich-henrique/datasets_soybeans_research/main/Gross%20Domestic%20Product.csv',skiprows=2)
consumer_price = pd.read_csv('https://raw.githubusercontent.com/friedrich-henrique/datasets_soybeans_research/main/Consumer%20Price%20Index.csv?token=GHSAT0AAAAAACAIZJ3ILPT5RHHYRPV7PCQCZCWB3UA', skiprows=2)


soy_trade.rename(columns={"1201 in 1000 USD ": "trade_value"}, inplace=True)
soy_trade['ln_trade_value'] = soy_trade['trade_value'].apply(lambda x: np.log(x))

flour_trade.rename(columns={"120810 in 1000 USD ": "trade_value", "ReporterISO3": "Source", "PartnerISO3" : "Target"}, inplace=True)
flour_trade['ln_trade_value'] = flour_trade['trade_value'].apply(lambda x: np.log(x))

country_size.drop([0,1,2], inplace=True)
country_size.columns = country_size.iloc[0]
country_size.drop([3], inplace=True)
country_size.drop(columns=['Indicator Name','Indicator Code', 'Country Name', 1960.0], inplace=True)

gdp_data.rename({'Unnamed: 0': 'Country Code'}, inplace=True, axis=1)
ex_rate.rename({'Unnamed: 0': 'Country Code'}, inplace=True, axis=1)
consumer_price.rename({'Unnamed: 0': 'Country Code'}, inplace=True, axis=1)

In [4]:
sizes = country_size.melt(id_vars=["Country Code"],
                  var_name="Year",
                  value_name="Area")

sizes["Year"] = sizes["Year"].apply(lambda x: int(x))

In [5]:
ex_rate = ex_rate.melt(id_vars=["Country Code"],
                  var_name="Year",
                  value_name="Exchange Rate")

ex_rate["Year"] = ex_rate["Year"].apply(lambda x: int(x))

In [6]:
consumer_price = consumer_price.melt(id_vars=["Country Code"],
    var_name="Year",
    value_name="Consumer Price Index")

consumer_price["Year"] = consumer_price["Year"].apply(lambda x: int(x))

In [7]:
def add_attributes(trade_table, sizes_table, attribute_name):
    trade_table = trade_table.copy()
    sizes_table = sizes_table.copy()
    sizes_table = sizes_table[sizes_table["Year"] >= trade_table["Year"].min()] 
    trade_table.reset_index(drop=True, inplace=True)
    sizes_table.reset_index(drop=True, inplace=True)
    count = 0
    progress = 0
   
    for row in range(len(trade_table)):
        source_area_slice = sizes_table[(sizes_table["Country Code"] == trade_table.iloc[row, 0]) & (sizes_table["Year"] == trade_table.loc[row]["Year"])]
        target_area_slice = sizes_table[(sizes_table["Country Code"] == trade_table.iloc[row, 1]) & (sizes_table["Year"] == trade_table.loc[row]["Year"])]
        if len(source_area_slice) > 0:
            trade_table.at[row, f"Source {attribute_name}"] = source_area_slice.iloc[0][attribute_name]
        else: 
            trade_table.at[row, f"Source {attribute_name}"] = None

        if len(target_area_slice) > 0:
            trade_table.at[row, f"Target {attribute_name}"] = target_area_slice.iloc[0][attribute_name]
        else:
            trade_table.at[row, f"Target {attribute_name}"] = None
        count += 1
        current_progress = count / len(trade_table) * 100
        if current_progress >= 1:
            if current_progress >= progress + 10:
                progress = int(current_progress // 10) * 10
                print(f"{progress}% done")
    return trade_table

In [8]:
data = add_attributes(flour_trade if table=="flour" else soy_trade, sizes, "Area")

10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done


In [9]:
data = add_attributes(data, ex_rate, "Exchange Rate")

10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done


In [10]:
# Subsitute nan of eu countries with EMU exchange rate
## EMU = European Monetary Union (Euro)
eu_countries = ['AUT', 'BEL', 'CYP', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'PRT', 'SVK', 'SVN', 'ESP']

for year in range(1999, data['Year'].max()+1):
    for country in eu_countries:
        ex_rate_value = ex_rate[(ex_rate["Year"] == year) & (ex_rate["Country Code"] == 'EMU')]['Exchange Rate'].values[0]
        data.loc[(data["Year"] == year) & (data["Source"].isin(eu_countries)), 'Source Exchange Rate'] = ex_rate_value
        data.loc[(data["Year"] == year) & (data["Target"].isin(eu_countries)), 'Target Exchange Rate'] = ex_rate_value

In [11]:
data['Source Exchange Rate'] = [0 if i is None else float(str(i).replace(",", "")) for i in data["Source Exchange Rate"]]
data['Target Exchange Rate'] = [0 if i is None else float(str(i).replace(",", "")) for i in data["Target Exchange Rate"]]

data["Ex Rate Source / Target"] = data["Source Exchange Rate"] / data["Target Exchange Rate"]
data["Ex Rate Target / Source"] = data["Target Exchange Rate"] / data["Source Exchange Rate"]

In [12]:
data = add_attributes(data, consumer_price, "Consumer Price Index")

10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done


In [13]:
data['Source Consumer Price Index'] = [0 if i is None else float(str(i).replace(",", "")) for i in data["Source Consumer Price Index"]]
data['Target Consumer Price Index'] = [0 if i is None else float(str(i).replace(",", "")) for i in data["Target Consumer Price Index"]]

data["exchange_rate ijt"] = data["Ex Rate Target / Source"] * (data["Source Consumer Price Index"] / data["Target Consumer Price Index"])

gdp_data = gdp_data.melt(id_vars=["Country Code"],
    var_name="Year",
    value_name="GDP")
gdp_data["Year"] = gdp_data["Year"].apply(lambda x: int(x))

In [14]:
data = add_attributes(data, gdp_data, "GDP")

10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done
100% done


In [15]:
data['Source GDP'] = [0 if i is None else float(str(i).replace(",", "")) for i in data["Source GDP"]]
data['Target GDP'] = [0 if i is None else float(str(i).replace(",", "")) for i in data["Target GDP"]]

data.replace(0, np.nan, inplace=True)

data["level of output ijt"] = (np.log(data["Source GDP"]) + np.log(data["Target GDP"])) / 2

In [16]:
data.to_csv(f"{table}_dataset.csv", index=False)